# Short-Term Financing Optimization

In [1]:
import numpy as np
import gurobipy as gp

### Initialize Constraint Matrix, Objective Vector, and Constraint Vector

In [2]:
obj = np.zeros(14)
obj[13] = 1

A = np.zeros((6,14))
#x1+y1-z1:
A[0,[0,5,8]]=[1,1,-1]

#x2 +y2 -1.01x1 +1.003z1 - z2:
A[1,1]=1;A[1,6]=1;A[1,0]=-1.01;A[1,8]=1.003;A[1,9]=-1

#x3 +y3 -1.01x2 +1.003z2 - z3:
A[2,2]=1;A[2,7]=1;A[2,1]=-1.01;A[2,9]=1.003;A[2,10]=-1

#x4 - 1.02y1 -1.01x3 +1.003z3 - z4:
A[3,3]=1;A[3,5]=-1.02;A[3,2]=-1.01;A[3,10]=1.003;A[3,11]=-1

#x5 - 1.02y2 -1.01x4 +1.003z4 - z5:
A[4,4]=1;A[4,6]=-1.02;A[4,3]=-1.01;A[4,11]=1.003;A[4,12]=-1

#   - 1.02y3 -1.01x5 +1.003z5 - z6:
A[5,7]=-1.02;A[5,4]=-1.01;A[5,12]=1.003;A[5,13]=-1

# xi <= 100
ub = np.array([100]*5+[np.Inf]*9)

#equality and inequality constraints
b=np.array([150,100,-200,200,-50,-300])
sense=np.array(["=","=","=","=","=","="])


### Solve the Model

In [4]:
stfMod = gp.Model()
stfMod_x = stfMod.addMVar(len(obj),ub=ub) # tell the model how many variables there are
# must define the variables before adding constraints because variables go into the constraints
stfMod_con = stfMod.addMConstrs(A, stfMod_x, sense, b) # NAME THE CONSTRAINTS!!! so we can get information about them later!
stfMod.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE) # add the objective to the model...we'll talk about the None and the 0

stfMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
stfMod.optimize()

Academic license - for non-commercial use only - expires 2022-08-28
Using license file C:\Users\Brandon\gurobi.lic


In [5]:
stfMod_x.x[0:5] # interest rate is high so we want to avoid borrowing from the bank

array([ 0.        , 50.98039216,  0.        ,  0.        ,  0.        ])

In [6]:
stfMod_x.x[5:8] # borrowing a lot from creditors to make up for the high bank rate

array([150.        ,  49.01960784, 203.43436358])

In [7]:
stfMod_x.x[8:14]

array([  0.        ,   0.        , 351.9441675 ,   0.        ,
         0.        ,  92.49694915])

In [ ]:
# in time period 6 we're left with $92.50
# in the problems statement we owe a total of $450 (negative cash flows)
# we will receive a total of $550 (positive cash flows)
# why aren't we left with $100 at time 6?
# we didn't have enough money to pay our early obligations
# we had to borrow money to satisfy our obligations and pay interest on it!